# Finding which frames are ESV in volume tracing data

## Load csv files

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data
tracings_df = pd.read_csv('../../dataset/EchoNet/VolumeTracings.csv')
file_info_df = pd.read_csv('../../dataset/EchoNet/FileInfo.csv')

In [2]:
tracings_df.head()

,FileName,X1,Y1,X2,Y2,Frame
0,0X100009310A3BD7FC.avi,51.260417,15.348958,64.932292,69.125000,46
1,0X100009310A3BD7FC.avi,50.037611,17.167841,53.367222,16.321330,46
2,0X100009310A3BD7FC.avi,49.157378,20.407629,57.090549,18.390722,46
3,0X100009310A3BD7FC.avi,48.538173,23.581055,59.997339,20.667707,46
4,0X100009310A3BD7FC.avi,47.918968,26.754480,62.904129,22.944693,46


In [3]:
file_info_df.head()

,FileName,Split,ESV,EDV
0,0X650881B7ADEF9668,Train,157,179
1,0X23B1641319A2AFA2,Train,50,66
2,0X21780197024E9D79,Train,61,76
3,0X304FEB38430F6379,Train,46,63
4,0X7FE48E5475ADE2E3,Train,119,94


In [4]:
train_df = file_info_df[file_info_df['Split'] == 'Train']
train_df

,FileName,Split,ESV,EDV
0,0X650881B7ADEF9668,Train,157,179
1,0X23B1641319A2AFA2,Train,50,66
2,0X21780197024E9D79,Train,61,76
3,0X304FEB38430F6379,Train,46,63
4,0X7FE48E5475ADE2E3,Train,119,94
...,...,...,...,...
6013,0X3BB59C8B26565746,Train,45,59
6014,0X39F5B35EA80E4D86,Train,80,95
6015,0X60AC441DE6762A5C,Train,127,98
6016,0X2989EFBB544B63BF,Train,61,84


## Convert tracing data to coordinates.

In [5]:
from tqdm import tqdm

coords = dict()
for n in tqdm(train_df['FileName']):
    name = f'{n}.avi'
    tracings = tracings_df[tracings_df['FileName'] == name]
    frames = tracings['Frame'].unique()
    coord = dict()
    for f in frames:
        t = tracings[tracings['Frame'] == f]
        points_l = list()
        points_r = list()
        for i in range(len(t)):
            c_1 = min(int(t.iloc[i]['X1'] + 0.5), 111)
            r_1 = min(int(t.iloc[i]['Y1'] + 0.5), 111)
            c_2 = min(int(t.iloc[i]['X2'] + 0.5), 111)
            r_2 = min(int(t.iloc[i]['Y2'] + 0.5), 111)
            if c_1 > c_2:
                c_1, c_2 = c_2, c_1
                r_1, r_2 = r_2, r_1
            points_l.append([c_1, r_1])
            points_r.append([c_2, r_2])
        points_l.sort(key=lambda x: x[1])
        points_r.sort(key=lambda x: x[1], reverse=True)
        points = points_l + points_r
        coord[f] = points
    coords[n] = coord

100%|██████████| 6018/6018 [02:46<00:00, 36.25it/s]


In [6]:
coords

{'0X650881B7ADEF9668': {157: [[48, 25],
   [47, 26],
   [47, 29],
   [47, 32],
   [47, 35],
   [48, 37],
   [49, 40],
   [49, 43],
   [50, 45],
   [51, 48],
   [52, 50],
   [53, 53],
   [53, 55],
   [54, 58],
   [54, 61],
   [54, 64],
   [54, 67],
   [54, 69],
   [53, 72],
   [53, 75],
   [55, 78],
   [66, 75],
   [77, 70],
   [78, 66],
   [79, 63],
   [79, 60],
   [79, 57],
   [79, 55],
   [79, 52],
   [79, 49],
   [78, 46],
   [77, 44],
   [76, 42],
   [75, 39],
   [73, 37],
   [71, 35],
   [69, 33],
   [67, 30],
   [65, 29],
   [62, 27],
   [52, 25],
   [59, 25]],
  179: [[54, 26],
   [53, 27],
   [51, 29],
   [51, 32],
   [50, 34],
   [50, 36],
   [50, 38],
   [50, 40],
   [50, 42],
   [51, 44],
   [52, 46],
   [53, 48],
   [53, 50],
   [53, 52],
   [53, 54],
   [53, 56],
   [52, 59],
   [51, 61],
   [50, 63],
   [49, 65],
   [49, 67],
   [58, 67],
   [69, 65],
   [69, 63],
   [69, 61],
   [69, 59],
   [69, 57],
   [69, 55],
   [69, 53],
   [69, 51],
   [69, 49],
   [68, 47],
   [6

## Find ESV, EDV

In [7]:
from shapely.geometry import Polygon

areas = dict()
for n, coord in coords.items():
    area = dict()
    frames = list(coord.keys())
    for f, points in coord.items():
        polygon = Polygon(points)
        area[f] = polygon.area
    if area[frames[0]] < area[frames[1]]:
        area = {'ESV': frames[0], 'EDV': frames[1]}
    else:
        area = {'ESV': frames[1], 'EDV': frames[0]}
    areas[n] = area
print(areas)

{'0X650881B7ADEF9668': {'ESV': 179, 'EDV': 157}, '0X23B1641319A2AFA2': {'ESV': 66, 'EDV': 50}, '0X21780197024E9D79': {'ESV': 76, 'EDV': 61}, '0X304FEB38430F6379': {'ESV': 63, 'EDV': 46}, '0X7FE48E5475ADE2E3': {'ESV': 94, 'EDV': 119}, '0X729CD3050F8450DF': {'ESV': 94, 'EDV': 82}, '0X30A55286F1F00DD9': {'ESV': 136, 'EDV': 120}, '0X5678F14272EC915': {'ESV': 86, 'EDV': 72}, '0X6FD03091458DAD1C': {'ESV': 57, 'EDV': 41}, '0X9A5EF4B2212EA30': {'ESV': 57, 'EDV': 44}, '0X605AAC45ACC879B4': {'ESV': 17, 'EDV': 4}, '0X73E20506F6BF4FD6': {'ESV': 113, 'EDV': 94}, '0X5C26378213CC3D5E': {'ESV': 61, 'EDV': 52}, '0X6200291BC8CEC0A8': {'ESV': 17, 'EDV': 0}, '0X7E2437FA9BB9A9BE': {'ESV': 17, 'EDV': 3}, '0X15DA8D60960ABB2B': {'ESV': 71, 'EDV': 53}, '0X2FE7BCA20D009E39': {'ESV': 77, 'EDV': 61}, '0X3A66D46DC689ADD2': {'ESV': 71, 'EDV': 52}, '0X5F7A3F7ECEF71001': {'ESV': 68, 'EDV': 52}, '0X1EDF77660FB2302C': {'ESV': 65, 'EDV': 47}, '0X7FBA0A353972E8CE': {'ESV': 54, 'EDV': 37}, '0X4E3F9006EAE3AF6A': {'ESV': 11

## Save csv

In [8]:
file_info_df['ESV'] = -1
file_info_df['EDV'] = -1
file_info_df.head()

,FileName,Split,ESV,EDV
0,0X650881B7ADEF9668,Train,-1,-1
1,0X23B1641319A2AFA2,Train,-1,-1
2,0X21780197024E9D79,Train,-1,-1
3,0X304FEB38430F6379,Train,-1,-1
4,0X7FE48E5475ADE2E3,Train,-1,-1


In [9]:
for name, area in areas.items():
    esv = area['ESV']
    edv = area['EDV']
    file_info_df.loc[file_info_df['FileName'] == name, 'ESV'] = esv
    file_info_df.loc[file_info_df['FileName'] == name, 'EDV'] = edv
file_info_df.head()

,FileName,Split,ESV,EDV
0,0X650881B7ADEF9668,Train,179,157
1,0X23B1641319A2AFA2,Train,66,50
2,0X21780197024E9D79,Train,76,61
3,0X304FEB38430F6379,Train,63,46
4,0X7FE48E5475ADE2E3,Train,94,119


In [10]:
file_info_df.tail()

,FileName,Split,ESV,EDV
10025,0X234005774F4CB5CD,Test,-1,-1
10026,0X2DC68261CBCC04AE,Test,-1,-1
10027,0X35291BE9AB90FB89,Test,-1,-1
10028,0X6C435C1B417FDE8A,Test,-1,-1
10029,0X5515B0BD077BE68A,Test,-1,-1


In [11]:
file_info_df.dtypes

FileName    object
Split       object
ESV          int64
EDV          int64
dtype: object

In [12]:
file_info_df.to_csv('../../dataset/EchoNet/FileInfo.csv', index=False)